In [2]:
from calitp_data_analysis.tables import tbls
from siuba import _, filter, count, collect, show_query
import pandas as pd
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/ntd/"

pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  

# What does the data look like?

In [3]:
# using siuba to read in data from warehouse.
ntd_info = (tbls.mart_ntd.dim_annual_ntd_agency_information() 
            >> filter(_._is_current == True) 
            >> collect()
           )

ntd_service = (tbls.mart_ntd.dim_annual_ntd_agency_service()
             >> collect()
            )

In [35]:
#join ntd_info to ntd_service to get uza name?

merge = pd.merge(ntd_service, 
                 ntd_info, 
                 how = "left",
                 on = ["ntd_id",
                       "agency_name",
                       "reporter_type",
                       "subrecipient_type",
                       "reporting_module",
                      ],
                 indicator = True
                )


KeyError: 'subrecipient_typereporting_module'

In [ ]:
display(
    merge.shape,
    merge["_merge"].value_counts(),
    list(merge.columns)
)
keep_cols = [
    "ntd_id",
    "agency_name",
    "mode",
    "tos",
    "reporter_type",
    "uza_name",
    "vehicles_passenger_cars_operated_in_maximum_service",
    "total_voms",
    "actual_vehicles_passenger_car_revenue_miles",
    "actual_vehicle_passenger_car_revenue_hours",
    "unlinked_passenger_trips__upt_"
    "_merge"
    
]

In [42]:
#dropped other columns
merge2 = merge[keep_cols]

In [45]:
#filter uza namne CA
# need to fill NaNs in uza name with 0, then merge on crosswalk.
merge2 = merge2[merge2["uza_name"].str.contains(", CA")]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
# reading in NTD ID crosswalk from GCS
crosswalk = pd.read_csv(
        f"{GCS_FILE_PATH}ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    ).rename(columns={"NTD ID": "ntd_id"})

In [11]:
crosswalk.columns

Index(['ntd_id', 'Legacy NTD ID', 'Agency', 'UZA Name', 'RTPA_open_data',
       'RTPA'],
      dtype='object')

In [12]:
# filtering ntd_service df by the ntd_id_list

ntd_service_ca = pd.merge(ntd_service, crosswalk, how="left", on="ntd_id",indicator=True)


In [15]:
ntd_service_ca["_merge"].value_counts()

left_only     11270
both           1735
right_only        0
Name: _merge, dtype: int64

In [13]:
# confirmed filtering worked.

display(
    ntd_service.shape,
    ntd_service_ca.shape
)


(13005, 43)

(13005, 49)

In [ ]:
# test aggregating by ntd_id to show VOMS
# expanded function to agg these columns by different groups (agency, mode, tos)
def group_agg(df, col):
    
    df = (df.groupby([col])
        .agg({
            "unlinked_passenger_trips__upt_":"sum",
            "vehicles_passenger_cars_operated_in_maximum_service":"max",
            "actual_vehicles_passenger_car_revenue_miles":"sum",
            "actual_vehicle_passenger_car_revenue_hours":"sum"
        }).reset_index()
       )
    return df

In [ ]:
# agg by agency name
group_agg(ntd_service_ca,"agency_name")

In [ ]:
# agg by mode
group_agg(ntd_service_ca,'mode')

In [ ]:
# agg by tos
group_agg(ntd_service_ca, "tos")